<a href="https://colab.research.google.com/github/pleasewaitinthequeue/sqlglot-lightning-talk/blob/main/sqlglot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem

I am tired of my old database and it costs too much.  I want to migrate to another type of database that is "better" for my business.  I have a bunch of SQL code from the old database, and I want to migrate it to another database.  How do I **translate** the code from **SQL Dialect A** ==> **SQL Dialect B**?

## What is a Dialect
Some of you are wondering what the heck I mean by dialect.  SQL is a standard right?  Shouldn't it be the same everywhere?

**ANSI-SQL** is a standard, but databases are usually paid products, and so software companies have a financial incentive to make their database do special things that are exclusive to that database.  Most databases will compile ANSI-SQL, but each database has expressions, functions, procedures, and flavor that is unique to that database.  Therefore, the syntax that each database accepts often needs to be specific to the brand of database.  Some brands lean heavily into custom syntax while others use the ANSI-SQL syntax, but have custom functions.  Some widely used SQL dialects are:  
* Microsoft = Transact-SQL (T-SQL)
* MySQL / MariaDB / PostgreSQL = (ANSI-SQL)
* Oracle = Procedural Language-SQL (PL-SQL)

*there are some really popular "Not Only SQL" databases that I left off of here for brevity [source](https://www.bairesdev.com/blog/most-popular-databases/)

## Solutions
* Burn Old DB Down and Build a New DB
* Go through the SQL code line by line Finding / Replacing keywords that aren't compatible
* Hire Consultants / Outsource
* Ask ChatGPT / Gemini / CoPilot
* Write a Python Program Leveraging existing Libraries

| Option | Pros | Cons |
| ------ | ------ | ------ |
| Burn it Down | Start Fresh | Lose Data?!!!! |
| Edit Code | Precise | Time Consuming |
| Hire Consultants | Quick | Expensive |
| Ask LLMs | Free or Cheap | Lacks Precision / Context |
| Write Python | Free or Cheap | Limits to Precision / Context |


# A Handful of Options
* sqlparse - (IR 2009) - non validating sql **parser** that does not understand dialects
* sqlfluff - (IR 2018) - dialect-flexible **parser/linter**
* sqlglot - (IR 2021) - **parser, transpiler, and optimizer** with dialect support
* sqltree - (In Development) - **parser/formatter** that supports limited dialects

Eliza Pan - 05/10/2023 - Open-source Python SQL parsers [(on Medium.com)](https://medium.com/@elizapan/open-source-python-sql-parsers-8dcfaf0c896a)
She has a series of articles on medium describing migration from Amazon Redshift to Trino DB (both are distributed No-SQL databases).  She leveraged **sqlglot** to help with these tasks.

# SQL Glot

SQLGlot is a no-dependency SQL parser, transpiler, optimizer, and engine. It can be used to format SQL or translate between 24 different dialects like DuckDB, Presto / Trino, Spark / Databricks, Snowflake, and BigQuery. It aims to read a wide variety of SQL inputs and output syntactically and semantically correct SQL in the targeted dialects.
* Project Website: https://sqlglot.com/sqlglot.html
* Code Repository: https://github.com/tobymao/sqlglot
* LinkedIn: https://www.linkedin.com/in/toby-mao/

## SQLglot is pythonic!?
**Mostly**....Lower level components such as the tokenization are handled by RUST.  According to github, rust accounts for 1.3% of the code or 6 files within this library at the time of writing.  RUST is mainly used during the tokenization / parsing process.

# Toby [Tobias] Mao [bio](https://g.co/kgs/2NVAz8D):
* Born 1988
* Hillsborough, CA
* B.A. in Computer Science from Northwestern University in 2011
* Number of jobs working at Tech firms since then.  Notably Netflix and AirBnB prior to founding his own analytics firm called [Tobiko Data](https://tobikodata.com/)

It was at these companies that he likely wanted to rip his own hair out working with all of the different types of databases.  Netflix and AirBnB leverage a number of traditional rdbms as well as no-sql databases.  Tobiko Data offers a cloud data management solution similar to DBT promising cloud based ETL/ELT.  Tag lines from his company's website follow:

> ### Meet Tobiko Cloud: data transformation without the waste.
> #### It's everything dbt™ wishes it was.
> *Tobiko Cloud is powered by SQLMesh to parse SQL and track column-level lineage. Instantly see the impact of your changes, and run only the updates you need.*



>[SQLmesh](https://sqlmesh.readthedocs.io/en/stable/) is a next-generation data transformation framework designed to ship data quickly, efficiently, and without error. Data teams can efficiently run and deploy data transformations written in SQL or Python with visibility and control at any size.

[Diagram](https://github.com/TobikoData/sqlmesh/blob/main/docs/readme/architecture_diagram.png?raw=true)

Let's back up to SQL Glot though and discuss:
Just what is a "Transpiler"?  And why do you need one?

The word itself "Transpile" is related to the word Compile.  

SQL or **Structured Query Language** goes back to the early days of computing.  Although ANSI-SQL is a standard that most all databases conform to, each database has flavor and smell to their syntax that while similar - it is difficult to take a query from one database and run it on another.

The reason people are making so much noise about this library is because of its ability to **translate** between one dialect and another - demonstrated below.

In [24]:
import sqlglot
from sqlglot import dialects

#I have a Query in Transact-SQL (TSQL)
query1 = """
select
  top 1 --this is unique to t-sql
  id,
  load_date,
  getdate() [todate], --this is unique to t-sql
  'mystring' [string]
from dbo.MyTable
"""

#I want to run it on my identical Oracle Database
#I send it in, specifying the Dialect it is Written in, and the Dialect I want it to be Transpiled To
tsql2oracle = sqlglot.transpile(sql=query1, read=dialects.TSQL, write='oracle')
#SQL Glot returns a List (that includes warnings and errors if there are any)
print(tsql2oracle[0])

#I can do it again, to get back to my original dialect
oracle2tsql = sqlglot.transpile(sql=tsql2oracle[0], read='oracle', write='tsql')
#SQL Glot returns the same query in a semantically similar form
print(oracle2tsql[0])


SELECT id, load_date, CURRENT_TIMESTAMP AS "todate" /* this is unique to t-sql */, 'mystring' AS "string" FROM dbo.MyTable FETCH FIRST 1 /* this is unique to t-sql */ ROWS ONLY
SELECT id, load_date, GETDATE() AS [todate] /* this is unique to t-sql */, 'mystring' AS [string] FROM dbo.MyTable FETCH FIRST 1 /* this is unique to t-sql */ ROWS ONLY


There's other stuff you can do too, such as providing a set of example tables, and then getting queries in a number of formats.

In [4]:
from sqlglot import select, condition

# Here we can create arbitrary bits of code and slap them together
where = condition("Load_Date between CURRENT_TIMESTAMP() - 1 and CURRENT_TIMESTAMP()").and_("id >= 10")
queryString = select("*").from_("MyTable").where(where).sql()
print('Sqlglot generated query in ANSI format')
print(queryString)
print('\n')
#and then we can transpile that!?  But hey, it didn't really do anything?!
output = sqlglot.transpile( sql=queryString, read=None, write='tsql' )
print('Transpiled to Transact-SQL')
print(output[0])
#the default dialect is ANSI SQL

Sqlglot generated query in ANSI format
SELECT * FROM MyTable WHERE Load_Date BETWEEN CURRENT_TIMESTAMP() - 1 AND CURRENT_TIMESTAMP() AND id >= 10


Transpiled to Transact-SQL
SELECT * FROM MyTable WHERE Load_Date BETWEEN GETDATE() - 1 AND GETDATE() AND id >= 10


*This sounds great but how does it actually do that?*

The secret sauce is in a concept call the **AST or Abstract Syntax Tree**.  ASTs are typically used in computer science to map out a [program flow](https://en.wikipedia.org/wiki/Abstract_syntax_tree#/media/File:Abstract_syntax_tree_for_Euclidean_algorithm.svg).  They are called **abstract** because the structural elements of the program are extracted from the source code and represented as a **tree**.

The value that **sqlglot** adds is to parse SQL queries into an AST, which can then be used to identify portions of the query that might be different from dialect to dialect.

SQLglot's documentation calls out three main parts of this library:
* **Tokenizer**: Converts raw code into a sequence of “tokens”, one for each word/symbol
* **Parser**: Converts sequence of tokens into an abstract syntax tree representing the semantics of the raw code
* **Generator**: Converts an abstract syntax tree into SQL code

In [5]:
from sqlglot import tokenize
#the tokenize method converts a query string into a list of expressions, categorizing the tokens.
sql = """
select
  bar.a,
  baz.b + 1 as b
from bar
join baz
on baz.a = bar.a
"""
tokens = tokenize(sql)
for token in tokens:
    print(token)


<Token token_type: TokenType.SELECT, text: select, line: 2, col: 6, start: 1, end: 6, comments: []>
<Token token_type: TokenType.VAR, text: bar, line: 3, col: 4, start: 10, end: 12, comments: []>
<Token token_type: TokenType.DOT, text: ., line: 3, col: 5, start: 13, end: 13, comments: []>
<Token token_type: TokenType.VAR, text: a, line: 3, col: 6, start: 14, end: 14, comments: []>
<Token token_type: TokenType.COMMA, text: ,, line: 3, col: 7, start: 15, end: 15, comments: []>
<Token token_type: TokenType.VAR, text: baz, line: 4, col: 4, start: 19, end: 21, comments: []>
<Token token_type: TokenType.DOT, text: ., line: 4, col: 5, start: 22, end: 22, comments: []>
<Token token_type: TokenType.VAR, text: b, line: 4, col: 6, start: 23, end: 23, comments: []>
<Token token_type: TokenType.PLUS, text: +, line: 4, col: 8, start: 25, end: 25, comments: []>
<Token token_type: TokenType.NUMBER, text: 1, line: 4, col: 10, start: 27, end: 27, comments: []>
<Token token_type: TokenType.ALIAS, text: a

In [6]:
from sqlglot import parse_one

#in the abstract syntax tree the tokens are nested
ast = parse_one(sql)

print(repr(ast))

Select(
  expressions=[
    Column(
      this=Identifier(this=a, quoted=False),
      table=Identifier(this=bar, quoted=False)),
    Alias(
      this=Add(
        this=Column(
          this=Identifier(this=b, quoted=False),
          table=Identifier(this=baz, quoted=False)),
        expression=Literal(this=1, is_string=False)),
      alias=Identifier(this=b, quoted=False))],
  from=From(
    this=Table(
      this=Identifier(this=bar, quoted=False))),
  joins=[
    Join(
      this=Table(
        this=Identifier(this=baz, quoted=False)),
      on=EQ(
        this=Column(
          this=Identifier(this=a, quoted=False),
          table=Identifier(this=baz, quoted=False)),
        expression=Column(
          this=Identifier(this=a, quoted=False),
          table=Identifier(this=bar, quoted=False))))])


SQLglot also includes an optimizer, that will check the AST against a set of standardized rules.  Some of these include:
* fully qualifying tables and columns
* simplifying
* normalizing
* unnesting subqueries

[***There are 17 validation rules in total.***](https://github.com/tobymao/sqlglot/tree/main/sqlglot/optimizer)

In [7]:
from sqlglot.optimizer import optimize

opt_ast = optimize(
    ast,
    schema={
        "bar":{"a":"int","b":"int","c":"int"},
        "baz":{"a":"int","b":"int","c":"int"},
    }
    )
print(repr(opt_ast))
print('\n')
print(opt_ast.sql())

Select(
  expressions=[
    Alias(
      this=Column(
        this=Identifier(this=a, quoted=True),
        table=Identifier(this=bar, quoted=True),
        _type=DataType(this=Type.INT, nested=False)),
      alias=Identifier(this=a, quoted=True, _type=DataType(this=Type.UNKNOWN)),
      _type=DataType(this=Type.INT, nested=False)),
    Alias(
      this=Add(
        this=Column(
          this=Identifier(this=b, quoted=True),
          table=Identifier(this=baz, quoted=True),
          _type=DataType(this=Type.INT, nested=False)),
        expression=Literal(this=1, is_string=False, _type=DataType(this=Type.INT)),
        _type=DataType(this=Type.INT)),
      alias=Identifier(this=b, quoted=True, _type=DataType(this=Type.UNKNOWN)),
      _type=DataType(this=Type.INT))],
  from=From(
    this=Table(
      this=Identifier(this=bar, quoted=True, _type=DataType(this=Type.UNKNOWN)),
      alias=TableAlias(
        this=Identifier(this=bar, quoted=True, _type=DataType(this=Type.UNKNOWN)),
  

SQLglot can be used for Optimization and Validation, provided that it is aware of the database schemas in question.


In [8]:
from sqlglot import dialects
tsql2oracle = sqlglot.transpile(sql=query1,read='tsql', write='oracle')
print(tsql2oracle[0])
print('\n')

oracle2tsql = sqlglot.transpile(sql=query1, read='oracle', write='tsql')
print(oracle2tsql[0])
print('\n')

SELECT id, load_date, CURRENT_TIMESTAMP AS "todate" /* this is unique to t-sql */, 'mystring' AS "string" FROM dbo.MyTable FETCH FIRST 1 /* this is unique to t-sql */ ROWS ONLY


SELECT TOP 1 /* this is unique to t-sql */ id, load_date, GETDATE()[todate] /* this is unique to t-sql */, 'mystring'[string] FROM dbo.MyTable




As you can see from the above, once we are in an ANSI SQL form, we don't always keep the spacing, and the comments are preserved as multiline comments.

*The github often uses the phrase **best effort basis** when describing the preservation of spacing and comments*.  

Still if you have **A LOT** of sql queries that need to go from **a sql dialect** ==> **b sql dialect**, this would not be a bad way to identify the parts that are problematic.

Another potential use from this library is enumerating the differences between two somewhat similar queries.  See below, we combine the **diff** method with the **parse_one** method to get a list of things we would need to do to change **sql a** ==> **sql b**

In [20]:
sqla = "select a, b from baz"
sqlb = "select b, c from bar join baz on baz.a = bar.a"

diff = sqlglot.diff( parse_one(sqla), parse_one(sqlb) )
print(diff)

[Remove(expression=Select(
  expressions=[
    Column(
      this=Identifier(this=a, quoted=False)),
    Column(
      this=Identifier(this=b, quoted=False))],
  from=From(
    this=Table(
      this=Identifier(this=baz, quoted=False))))), Remove(expression=Column(
  this=Identifier(this=a, quoted=False))), Remove(expression=From(
  this=Table(
    this=Identifier(this=baz, quoted=False)))), Insert(expression=Select(
  expressions=[
    Column(
      this=Identifier(this=b, quoted=False)),
    Column(
      this=Identifier(this=c, quoted=False))],
  from=From(
    this=Table(
      this=Identifier(this=bar, quoted=False))),
  joins=[
    Join(
      this=Table(
        this=Identifier(this=baz, quoted=False)),
      on=EQ(
        this=Column(
          this=Identifier(this=a, quoted=False),
          table=Identifier(this=baz, quoted=False)),
        expression=Column(
          this=Identifier(this=a, quoted=False),
          table=Identifier(this=bar, quoted=False))))])), Insert(exp

# I am working with a Database that is Esoteric or Not Super Popular
## How do I write my own dialect?

Dialects are just .py files that are imported at runtime.  You can see them in the github under [/dialects](https://github.com/tobymao/sqlglot/tree/main/sqlglot/dialects).  You can two one of the following:


1.   Contribute to SQLglot by writing Dialect Files
2.   Create without Sharing *(by writing your own dialect and inserting locally)*

Here are the kinds of things that are in the **tsql.py** dialect file:
* Date & Time Formatting *(literals and expressions)*
* Date & Number Defaults *(such as 1900-01-01)*
* Custom Database functions *(such as EOMONTH() or STRING_AGG())*
* Custom Formats for Common Table Expressions (CTEs) and Subqueries
* Data Type Mapping
* Special **TOKENS** that don't appear in other languages
* Database Specific Compiler Hints

## Advice:  
If you're going to try and write a custom dialect file, you should know that dialect REALLY WELL

## Takeaways:
**This library does not replace expertise!**






# More to Follow

At work we are eventually going to replace **Oracle** databases with **Snowflake** as a data warehousing system of record, and so eventually I will be able to do a follow up talk where I describe how it went.

**sqlglot** has support for oracle and snowflake systems, and so for queries and views there will be opportunity to use this library to save time.